Los primeros 4 bloques son includes, setup, descarga de archivos, etc.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import math

import seaborn as sns

%matplotlib inline

plt.style.use('default')
sns.set(style="whitegrid")
pd.options.display.float_format = '{:20,.2f}'.format

import warnings
warnings.filterwarnings('ignore')

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

NameError: ignored

In [ ]:
id1="17SvnFyBNyY018rxrRnOLGp9YaZfeRknp"
downloaded1 = drive.CreateFile({'id': id1})
downloaded1.GetContentFile('googleplaystore.csv')

id2="1DBCzFlJWzdVirjQmoyTaH3GekLTzrznm"
downloaded2 = drive.CreateFile({'id': id2})
downloaded2.GetContentFile('googleplaystore_user_reviews.csv')

Los siguientes bloques inicializan los dataframes.

El primero iniciliza `ps_apps`, que corresponden a los datos de apps del primer csv. En el mismo bloque se formatean las columnas a tipos más convenientes, El código es bastante autoexplicativo. La línea 10473 se salta porque está mal formateada.

El segundo bloque inicializa `ps_reviews` que contiene la información del csv de las reviews. Este tiene menos columnas así que no hay que hacer tantos casteos.

In [ ]:
ps_apps = pd.read_csv("googleplaystore.csv", skiprows=[10473])

# Simple casts to category
ps_apps["Type"] = ps_apps["Type"].astype("category")
ps_apps["Content Rating"] = ps_apps["Content Rating"].astype("category")
ps_apps["Android Ver"] = ps_apps["Android Ver"].astype("category")
ps_apps["Installs"] = ps_apps["Installs"].map(lambda x : int(x[:-1].replace(',', '')) if x != "0" else 0)

#La columna de fechas
ps_apps['Last Updated'] = pd.to_datetime(ps_apps['Last Updated'], format="%B %d, %Y")

#lambdas
ps_apps["Genres"] = ps_apps["Genres"].map(lambda x : x.split(";"))

ps_apps["Category"] = ps_apps["Category"].map(lambda x : x.replace("_", " ").title()) #likely unneccessary
ps_apps["Category"] = ps_apps["Category"].astype("category")

def size_to_float(sz): # convert size to number, bit too much for a lambda
  if sz == "Varies with device":
    return math.nan
  sz = sz[:-1]
  return float(sz)
ps_apps["Size"] = ps_apps["Size"].map(size_to_float)

In [ ]:
ps_reviews = pd.read_csv("googleplaystore_user_reviews.csv")
ps_reviews["Sentiment"] = ps_reviews["Sentiment"].astype("category")

El siguiente bloque grafica un **histograma** del score de polaridad de sentimiento, de todas las reviews para las cuales hay data en el primero, y de todas las reviews con score distinto de 0 en el segundo.

Los valores parecen seguir una distribución parecida a una normal centrada alrededor del 0.25, a excepción de un pico altísimo alrededor del 0.

Como se puede apreciar en el segundo gráfico, si ignoramos los valores que son exactamente 0, obtenemos una distribución mucho más parecida a una normal. Es altamente probable que haya habido un error a la hora de generar el dataset y se le haya asignado 0 erróneamente a varias reviews

In [ ]:
f, (hist_ax1, hist_ax2) = plt.subplots(nrows=2, figsize=(10,12))

hist_ax = sns.histplot(
    data=ps_reviews,
    x="Sentiment_Polarity",
    color='turquoise',
    bins=25,
    ax=hist_ax1
)
hist_ax = sns.histplot(
    data=ps_reviews[ps_reviews.Sentiment_Polarity != 0],
    x="Sentiment_Polarity",
    color='mediumturquoise',
    bins=25,
    ax=hist_ax2
)

hist_ax1.set_title("Sentiment Polarity in app reviews")
hist_ax2.set_title("Sentiment Polarity in app reviews (w/o 0 scores)")
hist_ax1.set_xlabel("Polarity")
hist_ax2.set_xlabel("Polarity")
hist_ax1.set_ylabel("Frequency")
hist_ax2.set_ylabel("Frequency")

f.savefig("hist.png", dpi=200)

El siguiente bloque grafica un **boxplot** de la distribución de tamaños de las apps por tipo (paga/gratis).

Ambas tienen una media bastante parecida (alrededor de los 18mb), pero las pagas están un poco más esparcidas y es más común que sean más pesadas (cabe aclarar que el sample sizze de apps pagas es mucho menor).

No se muestran los outliers porque hay varios que se salen tanto de los valores esperados que dificultan la lectura del gráfico.

In [ ]:
f, boxplot_ax = plt.subplots(figsize=(10,6))

sns.boxplot(
        data=ps_apps,
        x="Size",
        y="Type",
        boxprops={"facecolor": (.45, .85, .85, .8)},
        medianprops={"color": "mediumseagreen"},
        showfliers=False,
        ax=boxplot_ax
        )

boxplot_ax.set_title("Average size by app type (free/paid)")
boxplot_ax.set_xlabel("Size (Mb)")

f.savefig("boxplot.png", dpi=200)

El siguiente bloque genera un **heatmap** del rating promedio por categoria y rating de edad. "Adults only 18+" y "Unrated" se dejaron por fuera por falta de datos.

El rating promedio de todas las apps ronda los 4.3, así que tiene sentido que la gran mayoría de las categorías también tengan un promedio parecido.

Se puede apreciar que las apps rateadas para "Everyone" tienen ratings bastante pegados al promedio (son las más numeorsas, por lo que influyen más al promedio general), y las apps con rating "Mature 17+" tienen varios valores por encima y por debajo de la media (es más "polarizante")

In [ ]:
f, heatmap_ax = plt.subplots(figsize=(12,8))
heatmap_ax.ticklabel_format(style='plain', axis='both')
heatmap_ax.set_title("Ratings by Category and Content Rating")
heatmap_df = ps_apps.groupby(["Category", "Content Rating"])["Rating"].mean().unstack()
heatmap_ax.set_facecolor('darkslategrey')
heatmap_ax.grid(False)

sns.heatmap(
    heatmap_df.drop(["Adults only 18+", "Unrated"], axis=1),
    cmap="mako",
    linewidths=0.05,
    linecolor='lightgray',
    annot=True,
    ax=heatmap_ax
    )
f.savefig("heatmap.png", dpi=200)

El siguiente bloque genera un **Scatter plot** de la polaridad y subjetividad promedio de las reviews por app, junto con su recta de regresión.

Se observa que el promedio de polaridad está alrededor del 0.25 (como en el histograma) y el de subjetividad un poco por arriba del 0.5. También se aprecia una ligera correlación de a mayor polaridad, mayor subjetividad. Los puntos más grandes (las apps con más descargas) siguen más de cerca esta tendencia.

In [ ]:
df_aux = ps_reviews[ps_reviews.Sentiment_Polarity != 0][["App", "Sentiment_Polarity", "Sentiment_Subjectivity"]].groupby(["App"]).mean()
ps_apps.reset_index()
df_aux.reset_index()
ps_apps_ext = pd.merge(ps_apps, df_aux, on="App", how="left")

f, scatter_ax = plt.subplots(figsize=((12,8)))
sns.scatterplot(
    data=ps_apps_ext,
    x="Sentiment_Polarity",
    y="Sentiment_Subjectivity",
    hue="Type",
    style="Type",
    size="Installs",
    sizes=(20, 200),
    palette="Set2",
    ax=scatter_ax,
    legend="brief",
    zorder=10
)
sns.regplot(
    data=ps_apps_ext,
    x="Sentiment_Polarity",
    y="Sentiment_Subjectivity",
    scatter=False,
    color="teal",
    ax=scatter_ax
)

scatter_ax.set_title("Sentiment Polarity vs. Subjectivity")
scatter_ax.set_xlabel("Sentiment Polarity")
scatter_ax.set_ylabel("Sentiment Subjectivity")
scatter_ax.set_xbound(-1.0, 1.05)
scatter_ax.set_ybound(-0.05, 1.05)

#makes legend actually legible
leg=scatter_ax.legend()
leg.get_texts()[4].set_text("<200M")
leg.get_texts()[5].set_text("200-400M")
leg.get_texts()[6].set_text("400-600M")
leg.get_texts()[7].set_text("600-800M")
leg.get_texts()[8].set_text("800-1B")
leg.get_texts()[9].set_text(">1B")

f.savefig("scatter.png", dpi=200)